In [2]:
from scipy.integrate import odeint
import numpy as np

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

# A resistor-capacitor circuit

![A resistor-capacitor circuit](RCCircuit.png "A resistor-capacitor circuit")

Consider we are interested in the output voltage across the capacitor $C$ as $y(t)$. The current $i(t)$ is given by: 

$$i(t) = \frac{v_{in}(t)- y(t)}{R} $$

The current across capacitor $C$ is given by:

$$ i(t) = C \frac{dy}{dt} $$

Combining those two equations and annotating $ \frac{dy}{dt} $ as $ \dot y$ we can finally write: 

$$ RC \dot y + y = v_{in} $$

Let's simulate such a system subject to different input voltages $v_{in}(t)$. 

# Response to a single step functions

In [55]:
def UnitStep(t):
    return 1

def OutputVoltage(y, t, f, a):
    R = 10e3              # 10k ohm
    C = 100e-6            # 100 u Farad
    
    v = a*f(t)     # the input voltage
    
    yp = (1.0/(R*C))*(v-y)
    return yp

timestep = 0.001
tend = 7

t = np.arange(0.0, tend, timestep)                 # interval of time of interest   
v0 = 0.2                                            # Initial voltage value on the capacitor

a1, a2 = 1, 3.3                                             # amplitude of the input voltage

v1 = a1*np.ones(t.size)                             # a unit step to help when plotting
y1 = odeint(OutputVoltage, v0, t, args=(UnitStep,a1))

v2 = a2*np.ones(t.size)                             # 3.3 V step to help when plotting
y2 = odeint(OutputVoltage, v0, t, args=(UnitStep,a2))

In [56]:
p = figure(plot_width=600, plot_height=400, title="RC response to step functions")

# response to unit step
p.line(t,v1, line_color = 'green')
p.line(t, y1[:,0], line_color = 'blue')

# response to 3.3 V
p.line(t,v2, line_color = 'green')
p.line(t, y2[:,0], line_color = 'blue')

show(p)

### Note that we can see the property of *homogeneity*

# Response to sine waves

In [57]:
def SineInput(t):
    return np.sin(t)

t = np.arange(0.0, 2*tend, timestep)                 # interval of time of interest   

v0 = 0
a1 = 1
a2 = 0.5

v1 = a1*np.sin(t)                             # sine wave of amplitude 1 to help when plotting
y1 = odeint(OutputVoltage, v0, t, args=(SineInput,a1))


v2 = a2*np.sin(t)                             # sine wave of amplitude 0.5 to help when plotting
y2 = odeint(OutputVoltage, v0, t, args=(SineInput,a2))

p = figure(plot_width=600, plot_height=400, title="RC response to sine functions")

# response to as sine wave of amplitude 1
p.line(t,v1, line_color = 'green')
p.line(t, y1[:,0], line_color = 'green', line_dash='dotted', line_width=3)

# response to as sine wave of amplitude 0.5
p.line(t,v2, line_color = 'blue')
p.line(t, y2[:,0], line_color = 'blue', line_dash='dotted', line_width=3)

show(p)

# Response to exponential

In [58]:
def ExpInput(t):
    return np.exp(-t)

t = np.arange(0.0, 2*tend, timestep)                 # interval of time of interest   

v0 = 0
a1 = 2

v1 = a1*np.exp(-t)                                     # exponential wave of amplitude 2 to help when plotting
y1 = odeint(OutputVoltage, v0, t, args=(ExpInput,a1))

p = figure(plot_width=600, plot_height=400, title="RC response to an exponential function")

# response to as sine wave of amplitude 1
p.line(t,v1, line_color = 'blue')
p.line(t, y1[:,0], line_color = 'blue', line_dash='dotted', line_width=3)

show(p)

# Response to an input of two waveforms

In [46]:
def OutputVoltage2(y, t, f, g, a1, a2):
    R = 10e3              # 10k ohm
    C = 100e-6            # 100 u Farad
    
    v1 = a1*f(t) 
    v2 = a2*g(t)     
    v = v1 + v2           # the sum of 2 input voltages
    
    yp = (1.0/(R*C))*(v-y)
    return yp

def ExpSineInput(t):
    x1 = np.exp
    return np.exp(-t)


# Exponential function with amplitude 2
v0 = 0
a1 = 4

v1 = a1*np.exp(-t)                                     # exponential wave of amplitude 2 to help when plotting
y1 = odeint(OutputVoltage, v0, t, args=(ExpInput,a1))

# Sine wave with amplitude 0.5
a2 = 0.5

v2 = a2*np.sin(t)                             # sine wave of amplitude 1 to help when plotting
y2 = odeint(OutputVoltage, v0, t, args=(SineInput,a2))

vc = a1*np.exp(-t) + a2*np.sin(t)
yc = odeint(OutputVoltage2, v0, t, args=(ExpInput,SineInput, a1, a2))


p = figure(plot_width=600, plot_height=400)

# response to as sine wave of amplitude 1
p.line(t,v1+v2, line_color = 'blue')
p.line(t, y1[:,0]+y2[:,0], line_color = 'blue', line_dash='dotted', line_width=3)

p.line(t,vc, line_color = 'red', line_dash='dashdot', line_width=1)
p.line(t, yc[:,0], line_color = 'red', line_dash='dashdot', line_width=1)

show(p)

### This is the property of *superposition* for linear systems

# An example of a non linear system 
The last slide on Lecture 2 had the equation:

$$ T = MgL \sin(\theta) $$

Note: $\theta$ is the independent variable, so this system does not depend on time $t$ but on angle $\theta$.

In [13]:
M = 0.5            # half a kilogram
g = 9.81           # gravity value in MKS
L = 0.3            # 30 cm of arm length

theta = np.arange(0, 3*np.pi, 0.01)        # the range of interest for theta

ftheta = 0.5*np.sin(theta) + 2*np.cos(theta) #to try the principle of superposition
T = M*g*L*np.sin(ftheta)

p = figure(plot_width=600, plot_height=400)

# the input function in blue
p.line(theta,ftheta, line_color = 'blue')
# the torque output T
p.line(theta,T, line_color = 'red')

show(p)


### Note this response is not linear, so the function $ y = sin(x) $ is not linear. 